In [1]:
import mindspore as ms
ms.set_context(mode=0, device_target="Ascend", device_id=7)
# ms.set_context(mode=0, device_target="CPU")

In [2]:
from mindformers import GPT2LMHeadModel, GPT2Tokenizer, TextStreamer, TextIteratorStreamer, BatchTextIteratorStreamer
tok = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# streamer = TextStreamer(tok)
# iter_streamer = TextIteratorStreamer(tok, skip_prompt=True, use_spm=False, timeout=1.5)
batch_streamer = BatchTextIteratorStreamer(tok, skip_prompt=True, use_spm=False, timeout=60)

2023-09-07 23:41:19,661 - mindformers - INFO - config in the yaml file ./checkpoint_download/gpt2/gpt2.yaml are used for tokenizer building.
2023-09-07 23:41:19,682 - mindformers - WARNING - Can't find the tokenizer_config.json in the file_dict. The content of file_dict is : {}
2023-09-07 23:41:19,683 - mindformers - INFO - build tokenizer class name is: GPT2Tokenizer using args {'unk_token': '<|endoftext|>', 'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'vocab_file': './checkpoint_download/gpt2/vocab.json', 'merges_file': './checkpoint_download/gpt2/merges.txt'}.


[WARNING] ME(55328:139783019255360,MainProcess):2023-09-07-23:41:20.232.210 [mindspore/ops/primitive.py:228] The in_strategy of the operator in your network will not take effect in stand_alone mode. This means the the shard function called in the network is ignored. 
If you want to enable it, please use semi auto or auto parallel mode by context.set_auto_parallel_context(parallel_mode=ParallelMode.SEMI_AUTO_PARALLEL or context.set_auto_parallel_context(parallel_mode=ParallelMode.AUTO_PARALLEL)
[WARNING] ME(55328:139783019255360,MainProcess):2023-09-07-23:41:20.388.716 [mindspore/common/_decorator.py:40] 'DropoutGenMask' is deprecated from version 1.5 and will be removed in a future version, use 'ops.Dropout' instead.
[WARNING] ME(55328:139783019255360,MainProcess):2023-09-07-23:41:20.389.712 [mindspore/common/_decorator.py:40] 'DropoutDoMask' is deprecated from version 1.5 and will be removed in a future version, use 'ops.Dropout' instead.
[WARNING] ME(55328:139783019255360,MainProcess

2023-09-07 23:41:20,960 - mindformers - INFO - start to read the ckpt file: 497772028
2023-09-07 23:41:22,349 - mindformers - INFO - weights in ./checkpoint_download/gpt2/gpt2.ckpt are loaded
2023-09-07 23:41:22,356 - mindformers - INFO - model built successfully!


In [14]:
text2 = ["An increasing sequence: one,", 
         "I love Beijing, because",
         "hello world", 
         "the largest planet in the solar system is",
         "just write a english poem",
         "the highest mountain in the world is",
         "the capital of America is",
         "the ancester of human being is",
         "now I become the death,"]
inputs2 = tok(text2, max_length=8, padding='max_length')

In [15]:
inputs2['input_ids']

[[2025, 3649, 8379, 25, 530, 11, 50256, 50256],
 [40, 1842, 11618, 11, 780, 50256, 50256, 50256],
 [31373, 995, 50256, 50256, 50256, 50256, 50256, 50256],
 [1169, 4387, 5440, 287, 262, 6591, 1080, 318],
 [3137, 3551, 257, 46932, 21247, 50256, 50256, 50256],
 [1169, 4511, 8598, 287, 262, 995, 318, 50256],
 [1169, 3139, 286, 2253, 318, 50256, 50256, 50256],
 [1169, 281, 33187, 286, 1692, 852, 318, 50256],
 [2197, 314, 1716, 262, 1918, 11, 50256, 50256]]

In [16]:
input_batch = inputs2['input_ids']
bs = len(input_batch)

In [19]:
# 单线程
from threading import Thread
generation_kwargs = dict(
    input_ids=input_batch,
    streamer=batch_streamer,
    max_length=20,
    top_k=1
)

In [33]:
batch_streamer.clear()

In [34]:
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()

origin_inputs[0]: [ 2025  3649  8379    25   530    11 50256 50256]
origin_inputs[1]: [   40  1842 11618    11   780 50256 50256 50256]
origin_inputs[2]: [31373   995 50256 50256 50256 50256 50256 50256]
origin_inputs[3]: [1169 4387 5440  287  262 6591 1080  318]
origin_inputs[4]: [ 3137  3551   257 46932 21247 50256 50256 50256]
origin_inputs[5]: [ 1169  4511  8598   287   262   995   318 50256]
origin_inputs[6]: [ 1169  3139   286  2253   318 50256 50256 50256]
origin_inputs[7]: [ 1169   281 33187   286  1692   852   318 50256]
origin_inputs[8]: [ 2197   314  1716   262  1918    11 50256 50256]


In [35]:
output = [""] * bs
print("thread.start! ")
for new_text in batch_streamer:
    print("streamer_batch:", new_text)
    for i in range(bs):
        output[i] += new_text[i]
print(" ===== For loop over! ===== ")

thread.start! 
streamer_batch: [' ', ' ', '', ' ', '', ' ', ' ', ' ', ' ']
streamer_batch: ['', '', '.\n', 'about ', '.\n', 'the ', 'the ', 'the ', 'and ']
streamer_batch: ['two, ', "it's ", '\n', '', '\n', '', 'United ', 'most ', 'the ']
streamer_batch: ['', 'a ', '', '', '', '', 'States ', 'beautiful ', 'resurrection ']
streamer_batch: ['three, ', 'beautiful ', 'The ', '1.5 ', '', '', 'of ', 'thing ', 'of ']
streamer_batch: ['', '', 'first ', 'billion ', "I'm ", 'Himalayas, ', '', 'in ', 'the ']
streamer_batch: ['four, ', 'city. ', 'thing ', '', 'not ', 'which ', 'America, ', 'the ', '']
streamer_batch: ['', '', 'I ', '', 'sure ', 'is ', 'and ', '', 'dead.\n']
streamer_batch: ['five, ', "It's ", 'noticed ', 'light-years ', 'if ', 'the ', 'the ', 'world, ', '\n']
streamer_batch: ['', 'a ', 'was ', '', 'you ', 'highest ', 'United ', 'and ', '']
streamer_batch: ['six, ', 'beautiful ', 'that ', 'away.\n', 'can ', 'mountain ', 'States ', 'the ', 'The ']
streamer_batch: ['', '', 'the ', '\

In [36]:
# thread.join()

In [37]:
output

[' two, three, four, five, six, seven, eight,',
 " it's a beautiful city. It's a beautiful city. It's a",
 '.\n\nThe first thing I noticed was that the first thing I noticed was that the',
 ' about 1.5 billion light-years away.\n\n',
 ".\n\nI'm not sure if you can read it, but I",
 ' the Himalayas, which is the highest mountain in the world',
 ' the United States of America, and the United States of America is the United',
 ' the most beautiful thing in the world, and the most beautiful thing',
 ' and the resurrection of the dead.\n\nThe first thing I do']

In [20]:
# 不做线程
result = model.generate(input_batch, streamer=batch_streamer, max_length=20, top_k=1)
batch_streamer.tokenizer.decode(result)

origin_inputs[0]: [ 2025  3649  8379    25   530    11 50256 50256]
origin_inputs[1]: [   40  1842 11618    11   780 50256 50256 50256]
origin_inputs[2]: [31373   995 50256 50256 50256 50256 50256 50256]
origin_inputs[3]: [1169 4387 5440  287  262 6591 1080  318]
origin_inputs[4]: [ 2025  3649  8379    25   530    11 50256 50256]
origin_inputs[5]: [   40  1842 11618    11   780 50256 50256 50256]
origin_inputs[6]: [31373   995 50256 50256 50256 50256 50256 50256]
origin_inputs[7]: [1169 4387 5440  287  262 6591 1080  318]
origin_inputs[8]: [ 2025  3649  8379    25   530    11 50256 50256]
origin_inputs[9]: [   40  1842 11618    11   780 50256 50256 50256]
origin_inputs[10]: [31373   995 50256 50256 50256 50256 50256 50256]
origin_inputs[11]: [1169 4387 5440  287  262 6591 1080  318]
origin_inputs[12]: [ 2025  3649  8379    25   530    11 50256 50256]
origin_inputs[13]: [   40  1842 11618    11   780 50256 50256 50256]
origin_inputs[14]: [31373   995 50256 50256 50256 50256 50256 50256]

['An increasing sequence: one, two, three, four, five, six, seven, eight,',
 "I love Beijing, because it's a beautiful city. It's a beautiful city. It's a",
 'hello world.\n\nThe first thing I noticed was that the first thing I noticed was that the',
 'the largest planet in the solar system is about 1.5 billion light-years away.\n\n',
 'An increasing sequence: one, two, three, four, five, six, seven, eight,',
 "I love Beijing, because it's a beautiful city. It's a beautiful city. It's a",
 'hello world.\n\nThe first thing I noticed was that the first thing I noticed was that the',
 'the largest planet in the solar system is about 1.5 billion light-years away.\n\n',
 'An increasing sequence: one, two, three, four, five, six, seven, eight,',
 "I love Beijing, because it's a beautiful city. It's a beautiful city. It's a",
 'hello world.\n\nThe first thing I noticed was that the first thing I noticed was that the',
 'the largest planet in the solar system is about 1.5 billion light-years a

In [21]:
output = [""] * bs
for new_text_batch in batch_streamer:
    print("streamer iter:", new_text_batch)
    for i in range(bs):
        output[i] += new_text_batch[i]

streamer iter: [' ', ' ', '', ' ', ' ', ' ', '', ' ', ' ', ' ', '', ' ', ' ', ' ', '', ' ', ' ', ' ', '', ' ', ' ', ' ', '', ' ', ' ', ' ', '', ' ', ' ', ' ', '', ' ', ' ', ' ', '', ' ', ' ', ' ', '', ' ']
streamer iter: ['', '', '.\n', 'about ', '', '', '.\n', 'about ', '', '', '.\n', 'about ', '', '', '.\n', 'about ', '', '', '.\n', 'about ', '', '', '.\n', 'about ', '', '', '.\n', 'about ', '', '', '.\n', 'about ', '', '', '.\n', 'about ', '', '', '.\n', 'about ']
streamer iter: ['two, ', "it's ", '\n', '', 'two, ', "it's ", '\n', '', 'two, ', "it's ", '\n', '', 'two, ', "it's ", '\n', '', 'two, ', "it's ", '\n', '', 'two, ', "it's ", '\n', '', 'two, ', "it's ", '\n', '', 'two, ', "it's ", '\n', '', 'two, ', "it's ", '\n', '', 'two, ', "it's ", '\n', '']
streamer iter: ['', 'a ', '', '', '', 'a ', '', '', '', 'a ', '', '', '', 'a ', '', '', '', 'a ', '', '', '', 'a ', '', '', '', 'a ', '', '', '', 'a ', '', '', '', 'a ', '', '', '', 'a ', '', '']
streamer iter: ['three, ', 'beautifu

In [22]:
output

[' two, three, four, five, six, seven, eight,',
 " it's a beautiful city. It's a beautiful city. It's a",
 '.\n\nThe first thing I noticed was that the first thing I noticed was that the',
 ' about 1.5 billion light-years away.\n\n',
 ' two, three, four, five, six, seven, eight,',
 " it's a beautiful city. It's a beautiful city. It's a",
 '.\n\nThe first thing I noticed was that the first thing I noticed was that the',
 ' about 1.5 billion light-years away.\n\n',
 ' two, three, four, five, six, seven, eight,',
 " it's a beautiful city. It's a beautiful city. It's a",
 '.\n\nThe first thing I noticed was that the first thing I noticed was that the',
 ' about 1.5 billion light-years away.\n\n',
 ' two, three, four, five, six, seven, eight,',
 " it's a beautiful city. It's a beautiful city. It's a",
 '.\n\nThe first thing I noticed was that the first thing I noticed was that the',
 ' about 1.5 billion light-years away.\n\n',
 ' two, three, four, five, six, seven, eight,',
 " it's a beauti

In [14]:
batch_streamer.clear()

In [62]:
# streamer1 = {1: "as", 2: "asd", 3: "21", 4: "asz"}
# streamer2 = {1: "qweq", 2: "zxc", 3: "13"}
# streamers = {1: iter(streamer1.values()), 2: iter(streamer2.values())}

In [21]:
# iter_batch_out = {}
# is_done = True
# for i, streamer in streamers.items():
#     try:
#         iter_batch_out[i] = next(streamer)
#         is_done = False
#     except StopIteration:
#         iter_batch_out[i] = ""
# if is_done:
#     raise StopIteration